In [1]:
import matplotlib as plt 
import numpy as np 
from matplotlib import colors
from PIL import Image, ImageDraw
import random
import pandas as pd
from collections import defaultdict


In [2]:
def createList(r1, r2):
        if (r1 == r2):
            return r1
        else:
            res = []
            while(r1 != r2):
                res.append(r1)
                r1 += 1
                if (r1 > 50):
                    r1 = r1-50
            res.append(r2)
        return res 

In [3]:
class Graph():
    
    def __init__(self):
        
        #create graph
        self.graph = {}
        for i in range(1,51):
            if i-1 == 0:
                self.graph[i] = [2, 50]
            elif i+1 == 51:
                self.graph[i] = [1, 49]
            else:
                self.graph[i] = [i+1, i-1]
        
        #create list of nodes
        self.nodes = createList(1, 50)
                
    def addEdge(self):
        boo1 = True
        while boo1:
            if len(self.nodes) == 0:
                boo1 = False
                break
            val = random.choice(self.nodes)
            self.nodes.remove(val)
            valstart = val-5
            valend = val+5
            if (valstart < 1):
                valstart = valstart+50
            if (valend > 50):
                valend = valend-50
            val_nodes = createList(valstart, valend)
            if(val == 1):
                val_nodes.remove(50)
            elif(val == 50):
                val_nodes.remove(1)
            else:
                val_nodes.remove(val-1)
                val_nodes.remove(val+1)
            boo2 = False
            while len(val_nodes) > 0:
                newnode = random.choice(val_nodes)
                if newnode in self.nodes:
                    self.nodes.remove(newnode)
                    self.graph[val].append(newnode)
                    self.graph[newnode].append(val)
                    boo2 = True
                    break
                else:
                    val_nodes.remove(newnode)
                    
            if boo2:
                break
        if boo1 == False:
            return False
        else:
            return True

In [4]:
#start position, end position, backwards path to take
#returns path
def BFS(start, end, graph):
 
    # a queue of the vertices whose path is to be scanned
    # boolean array for checking if node has been visited at least once or not
    # all moves in path set to -1
    queue = []
    visited = [False for i in range(51)]
    back_path = [-1 for i in range(51)]
    
    # start is visited and added to the queue
    visited[start] = True
    queue.append(start)

    # BFS algorithm - look at all neighbors first
    while (len(queue) != 0):
        u = queue[0]
        queue.pop(0)
        for i in range(len(graph[u])):
            if (visited[graph[u][i]] == False):
                visited[graph[u][i]] = True
                back_path[graph[u][i]] = u
                queue.append(graph[u][i])

                # We stop BFS when we find
                # destination.
                if (graph[u][i] == end):
                    path = []
                    crawl = end
                    path.append(crawl)
                    
                    #reversing back_path and removing -1 from nodes that are not in the path to get full path
                    while (back_path[crawl] != -1):
                        path.insert(0, back_path[crawl])
                        crawl = back_path[crawl]
                    return path

    return [end]

In [5]:
class Predator():
    
    def __init__(self, agentloc):
        self.pos = random.randint(1,50)
        #making sure that predator doesn't start where the agent is
        while self.pos == agentloc:
            self.pos = random.randint(1,50)
        
    def pred_move(self, g, agentpos):
        
        #getting all next possible moves of predator
        neighbors = g.graph[self.pos]
        path = []
        shortest_path = []
        path_length = 50
        #finding shortest path to agent using BFS
        for neighbor in neighbors:
            path = BFS(neighbor, agentpos, g.graph)
            
            #saving shortest path(s)
            if(len(path) <= path_length):
                if(len(path) == path_length):
                    shortest_path.append(path)
                else:
                    shortest_path = []
                    shortest_path.append(path)
                    path_length = len(path)
        
        #randomly picking path of the shortest paths and moving to next neighbor
        new_path = random.randint(0,(len(shortest_path)-1))
        self.pos = shortest_path[new_path][0]
        return self

In [6]:
class Prey():
    
    def __init__(self, agentloc):
        self.pos = random.randint(1,50)
        #making sure prey doesn't start where agent is
        while self.pos == agentloc:
            self.pos = random.randint(1,50)
        
    def prey_move(self, g):
        prey_loc_neighbours = [self.pos]
        for i in g.graph[self.pos]:
            prey_loc_neighbours.append(i)
        #randomly choose neighbor to move to (or staying put)
        self.pos = random.choice(prey_loc_neighbours)
        return self

In [7]:
def Agent3(g):
    agent_pos = random.randint(1,50)
    predator_3 = Predator(agent_pos)
    prey_3 = Prey(agent_pos)
    fail = 0
    steps = 0
    path = []
    path.append(agent_pos)

    belief_vector =  [(1/50)] * 50    #initializing belief-states
    #we know position of agent at first timestep
    for i in range(len(belief_vector)): #making probability of predator being in all other nodes 0
        prey_not = 1-belief_vector[agent_pos-1] #obviously prey isnt at agents position so get probability of prey not being in that position
        belief_vector[agent_pos-1] = 0 #making belief of prey in agents node 0
        for i in range(len(belief_vector)):
                belief_vector[i] = belief_vector[i]/prey_not #normalizing
    
    #print(belief_vector)
    
    while(fail == 0 and steps <= 1000):
        max_prob = max(belief_vector)
        #print(max_prob)
        #getting max probability value in belief vector
        list_of_positions_max_prob = []
        for itr in range(len(belief_vector)): #making a list of positions which have the highest probability of having the prey
            if(belief_vector[itr] == max_prob):
                list_of_positions_max_prob.append(itr+1)
        possible_prey_pos = random.choice(list_of_positions_max_prob) #randomly selecting one of those positions and surveying it
        #print(possible_pred_pos)
        
        #survey and consequent belief update
        if possible_prey_pos == prey_3.pos: #if we survey node and find prey
            for i in range(len(belief_vector)): #making probability of prey being in all other nodes 0
                if i == prey_3.pos-1:
                    belief_vector[i] = 1
                else:
                    belief_vector[i] = 0
        else:#incase we dont find, make probability at that position 0 and divide entire array by probability of prey not being in that position
            prey_not = 1-max_prob 
            belief_vector[possible_prey_pos-1] = 0
            for i in range(len(belief_vector)):
                    belief_vector[i] = belief_vector[i]/prey_not

        #print(str(belief_vector) + ' first update during timestep')
        #print(str(sum(belief_vector)) + " Update 1 Timestep: " + str(steps))
        #print(belief_vector)
    #    print(sum(belief_vector))
    #    if (sum(belief_vector) > 1.01 or sum(belief_vector) < 0.99):
    #        fail_list.append((1,steps))

        """quasi_1 = sum(belief_vector)
        #print(quasi)
        for i in range(len(belief_vector)):
            belief_vector[i] /= quasi_1""" #to ensure that belief vector sums up to 1, leaving no margin for error

        list_of_positions_max_chance = []
        for itr in range(len(belief_vector)): #making a list of positions which now have the highest probability of having the prey
            if(belief_vector[itr] == max(belief_vector)):
                list_of_positions_max_chance.append(itr+1)
        predicted_prey_pos = random.choice(list_of_positions_max_chance) #randomly selecting one of those positions and assuming it as prey position

        #Once positions are decided, moving according to Agent 1
        neighbors = g.graph[agent_pos]
        #neighbors.append(agent_pos)
        # moves = list of tuples - each path has a tuple with distance to prey, a distance to predator, and 
        # the neighbor they would move to for that path
        moves_1 = []
        moves_2 = []
        moves_3 = []
        moves_4 = []
        moves_5 = []
        moves_6 = []
        path_to_prey = []
        path_to_predator = []
        curr_to_pred = len(BFS(agent_pos, predator_3.pos, g.graph))
        curr_to_prey = len(BFS(agent_pos, predicted_prey_pos, g.graph))

            
        for neighbor in neighbors:
            
            # shortest path from neighbor to prey and shortest path from neighbor to predator
            path_to_prey = BFS(neighbor, predicted_prey_pos, g.graph)
            path_to_predator = BFS(neighbor, predator_3.pos, g.graph)
            
            #print("Prey and Predator Moves")
            #print(path_to_prey)
            #print(path_to_predator)
            if (len(path_to_prey) < curr_to_prey and len(path_to_predator) > curr_to_pred):
                tup = (len(path_to_prey), len(path_to_predator), neighbor)
                moves_1.append(tup)
            
            elif (len(path_to_prey) < curr_to_prey and len(path_to_predator) == curr_to_pred):
                tup = (len(path_to_prey), len(path_to_predator), neighbor)
                moves_2.append(tup)

            elif (len(path_to_prey) == curr_to_prey and len(path_to_predator) > curr_to_pred):
                tup = (len(path_to_prey), len(path_to_predator), neighbor)
                moves_3.append(tup)

            elif (len(path_to_prey) == curr_to_prey and len(path_to_predator) == curr_to_pred):
                tup = (len(path_to_prey), len(path_to_predator), neighbor)
                moves_4.append(tup)

            elif (len(path_to_predator) > curr_to_pred):
                tup = (len(path_to_prey), len(path_to_predator), neighbor)
                moves_5.append(tup)

            elif (len(path_to_predator) == curr_to_pred):
                tup = (len(path_to_prey), len(path_to_predator), neighbor)
                moves_6.append(tup)          

        
        #print()
        if(len(moves_1) > 0):
            moves_sorted = sorted(moves_1, key=lambda t: (t[0], -t[1]))
            #print(moves_sorted)
            move_choice=[]
            for move in moves_sorted: #to check if there are multiple /best/ neighbours which have the same distance to prey and predator
                if (moves_sorted[0][0] == move[0] and moves_sorted[0][1] == move[1]):
                    move_choice.append(move)
            neighbour_choice = random.choice(move_choice) #selecting from these multiple neighbours at random
            agent_pos = neighbour_choice[2]

        elif(len(moves_2) > 0):
            moves_sorted = sorted(moves_2, key=lambda t: (t[0], -t[1]))
            #print(moves_sorted)
            move_choice=[]
            for move in moves_sorted: #to check if there are multiple /best/ neighbours which have the same distance to prey and predator
                if (moves_sorted[0][0] == move[0] and moves_sorted[0][1] == move[1]):
                    move_choice.append(move)
            neighbour_choice = random.choice(move_choice) #selecting from these multiple neighbours at random
            agent_pos = neighbour_choice[2]

        elif(len(moves_3) > 0):
            moves_sorted = sorted(moves_3, key=lambda t: (t[0], -t[1]))
            #print(moves_sorted)
            move_choice=[]
            for move in moves_sorted: #to check if there are multiple /best/ neighbours which have the same distance to prey and predator
                if (moves_sorted[0][0] == move[0] and moves_sorted[0][1] == move[1]):
                    move_choice.append(move)
            neighbour_choice = random.choice(move_choice) #selecting from these multiple neighbours at random
            agent_pos = neighbour_choice[2]

        elif(len(moves_4) > 0):
            moves_sorted = sorted(moves_4, key=lambda t: (t[0], -t[1]))
            #print(moves_sorted)
            move_choice=[]
            for move in moves_sorted: #to check if there are multiple /best/ neighbours which have the same distance to prey and predator
                if (moves_sorted[0][0] == move[0] and moves_sorted[0][1] == move[1]):
                    move_choice.append(move)
            neighbour_choice = random.choice(move_choice) #selecting from these multiple neighbours at random
            agent_pos = neighbour_choice[2]

        elif(len(moves_5) > 0):
            moves_sorted = sorted(moves_5, key=lambda t: (-t[1],t[0]))
            #print(moves_sorted)
            move_choice=[]
            for move in moves_sorted: #to check if there are multiple /best/ neighbours which have the same distance to prey and predator
                if (moves_sorted[0][0] == move[0] and moves_sorted[0][1] == move[1]):
                    move_choice.append(move)
            neighbour_choice = random.choice(move_choice) #selecting from these multiple neighbours at random
            agent_pos = neighbour_choice[2]

        elif(len(moves_6) > 0):
            moves_sorted = sorted(moves_6, key=lambda t: (-t[1], t[0]))
            #print(moves_sorted)
            move_choice=[]
            for move in moves_sorted: #to check if there are multiple /best/ neighbours which have the same distance to prey and predator
                if (moves_sorted[0][0] == move[0] and moves_sorted[0][1] == move[1]):
                    move_choice.append(move)
            neighbour_choice = random.choice(move_choice) #selecting from these multiple neighbours at random
            agent_pos = neighbour_choice[2]
        
        path.append(agent_pos)

        steps += 1

        if (agent_pos == prey_3.pos): #if agent has already moved to location of prey
            break
        else: #if prey not caught, treat as survey and perform another belief update
            prey_not = 1-belief_vector[agent_pos-1] #incase we dont find, make probability at that position 0 and divide entire array by probability of prey not being in that position
            belief_vector[agent_pos-1] = 0
            for i in range(len(belief_vector)):
                    belief_vector[i] = belief_vector[i]/prey_not
            
        #print(str(sum(belief_vector)) + " Update 2 Timestep: " + str(steps))
        #print(str(belief_vector) + ' first update during timestep')
        #print(belief_vector)
    #    print(sum(belief_vector))        
    #    if (sum(belief_vector) > 1.01 or sum(belief_vector) < 0.99):
    #        fail_list.append((2,steps))        
        #print(sum(belief_vector))
        """quasi_3 = sum(belief_vector)
        #print(quasi)
        for i in range(len(belief_vector)):
            belief_vector[i] /= quasi_3 #to ensure that belief vector sums up to 1, leaving no margin for error"""
            

        #move predator and prey
        prey_3.prey_move(g)

        if (prey_3.pos == agent_pos): #if prey luckily moves to location of agent
            break

        predator_3.pred_move(g, agent_pos)

        if(predator_3.pos == agent_pos): #if predator catches agent
            fail = 1
            break
        
        #belief update for next timestep
        temp_belief_vector = belief_vector.copy() #making a temporary belief vector for correct calculations
        for i in range(len(belief_vector)):
            mag_sum =  temp_belief_vector[i]/(len(g.graph[i+1])+1) #probability of prey staying in same position
            i_neighbours = g.graph[i+1] #getting neighbours of current position
            #print(g.graph)
            #print(i+1)
            #print(i_neighbours)
            for bloke in i_neighbours:
                mag_sum += (temp_belief_vector[bloke-1]/(len(g.graph[bloke])+1)) #calculating probability of prey moving to node i if it was in node bloke
            belief_vector[i] = mag_sum #assigning probability
        #print(str(belief_vector) + ' last update before next timestep')
        #print(str(sum(belief_vector)) + " Update 3 Timestep: " + str(steps))
        #print(belief_vector)
        #print(sum(belief_vector))        
        #if (sum(belief_vector) > 1.01 or sum(belief_vector) < 0.99):
        #    fail_list.append((3,steps))        
        """quasi_2 = sum(belief_vector)
        print(quasi_2)
        for i in range(len(belief_vector)):
            belief_vector[i] /= quasi_2"""
        #print(belief_vector)

    #if (len(fail_list) > 0):
    #    print("Fail List: " + str(fail_list))

    if(steps > 1000):
            fail = 2     
    return fail, path

In [8]:
wins = 0
losses = 0
timeout = 0
times = 0
win_dist = []
loss_dist = []

while times < 3000:
    g = Graph()
    graph = g.addEdge()
    while graph:
        graph = g.addEdge()
    res, path = Agent3(g)
    
    if(res == 0):
        wins += 1
        win_dist.append(len(path))
    elif(res == 1):
        losses += 1
        loss_dist.append(len(path))
    else:
        timeout += 1
    times += 1
    
print('Number of wins: ' + str(wins))
print("Max Steps Taken During a Win: " + str(max(win_dist)-1))
print("Average Steps Taken During a Win: " + str((sum(win_dist)/wins)-1))
print('Number of losses: ' + str(losses))
print("Max Steps Taken During a Loss: " + str(max(loss_dist)-1))
print("Average Steps Taken During a Loss: " + str((sum(loss_dist)/losses)-1))
print('Number of timeouts: ' + str(timeout))
print("Success Rate: " + str((wins/times)))

Number of wins: 2479
Max Steps Taken During a Win: 113
Average Steps Taken During a Win: 21.503832190399354
Number of losses: 521
Max Steps Taken During a Loss: 138
Average Steps Taken During a Loss: 21.201535508637235
Number of timeouts: 0
Success Rate: 0.8263333333333334


In [9]:
def Agent4(g):
    agent_pos = random.randint(1,50)
    predator_4 = Predator(agent_pos)
    prey_4 = Prey(agent_pos)
    fail = 0
    steps = 0
    path = []
    path.append(agent_pos)
    #fail_list = []

    belief_vector =  [(1/50)] * 50    #initializing belief-states
    #we know position of agent at first timestep
    for i in range(len(belief_vector)): #making probability of prey being in all other nodes 0
        prey_not = 1-belief_vector[agent_pos-1] #obviously no prey at agent position, make probability at that position 0 and divide entire array by probability of prey not being in that position
        belief_vector[agent_pos-1] = 0
        for i in range(len(belief_vector)):
            belief_vector[i] = belief_vector[i]/prey_not
    
    #print(belief_vector)
    
    while(fail == 0 and steps <= 1000):
        max_prob = max(belief_vector)
        #print(max_prob)
        #getting max probability value in belief vector
        list_of_positions_max_prob = []
        for itr in range(len(belief_vector)): #making a list of positions which have the highest probability of having the prey
            if(belief_vector[itr] == max_prob):
                list_of_positions_max_prob.append(itr+1)
        possible_prey_pos = random.choice(list_of_positions_max_prob) #randomly selecting one of those positions and surveying it
        #print(possible_pred_pos)
        
        #survey and consequent belief update
        if possible_prey_pos == prey_4.pos: #if we survey node and find prey
            for i in range(len(belief_vector)): #making probability of prey being in all other nodes 0
                if i == prey_4.pos-1:
                    belief_vector[i] = 1
                else:
                    belief_vector[i] = 0
        else:
            prey_not = 1-max_prob #incase we dont find, make probability at that position 0 and divide entire array by probability of prey not being in that position
            belief_vector[possible_prey_pos-1] = 0
            for i in range(len(belief_vector)):
                belief_vector[i] = belief_vector[i]/prey_not

        #print(str(belief_vector) + ' first update during timestep')
        #print(str(sum(belief_vector)) + " Update 1 Timestep: " + str(steps))
        #print(belief_vector)
        #if (sum(belief_vector) > 1.01 or sum(belief_vector) < 0.99):
        #    fail_list.append((1,steps))

        """quasi_1 = sum(belief_vector)
        #print(quasi)
        for i in range(len(belief_vector)):
            belief_vector[i] /= quasi_1""" #to ensure that belief vector sums up to 1, leaving no margin for error

        list_of_positions_max_chance = []
        for itr in range(len(belief_vector)): #making a list of positions which now have the highest probability of having the prey
            if(belief_vector[itr] == max(belief_vector)):
                list_of_positions_max_chance.append(itr+1)
        predicted_prey_pos = random.choice(list_of_positions_max_chance) #randomly selecting one of those positions and assuming it as prey position

        #after deciding position of prey, applying Agent 2's rules all the way

        neighbors = g.graph[agent_pos]
        #neighbors.append(agent_pos)
        # moves = list of tuples - each path has a tuple with distance to prey, a distance to predator, and 
        # the neighbor they would move to for that path
        all_possible_moves = []
        amazing_moves = []
        path_to_prey = []
        path_to_pred = []
        curr_dist_to_pred = len(BFS(agent_pos, predator_4.pos, g.graph))
        curr_dist_to_prey = len(BFS(agent_pos, predicted_prey_pos, g.graph))
        all_possible_moves = [(curr_dist_to_prey, curr_dist_to_pred, agent_pos)]        

        
        for neighbour in neighbors:
            path_to_prey = BFS(neighbour, predicted_prey_pos, g.graph)
            path_to_pred = BFS(neighbour, predator_4.pos, g.graph)

            #keeping all moves handy
            tup2 = (len(path_to_prey), len(path_to_pred), neighbour)
            all_possible_moves.append(tup2)
            
            #making list of moves which only satisfy 1st condition of Agent 1
            if(len(path_to_pred) > curr_dist_to_pred and len(path_to_prey) < curr_dist_to_prey):
                tup3 = (len(path_to_prey), len(path_to_pred), neighbour)
                amazing_moves.append(tup3)                


        if(len(amazing_moves) > 0):             
            moves_sorted = sorted(amazing_moves, key=lambda t: (t[0], -t[1]))
            move_choice=[]
            for move in moves_sorted: #to check if there are multiple /best/ neighbours which have the same distance to prey and predator
                if (moves_sorted[0][0] == move[0] and moves_sorted[0][1] == move[1]): 
                    move_choice.append(move)
            neighbour_choice = random.choice(move_choice) #selecting from these multiple neighbours at random
            agent_pos = neighbour_choice[2]


        else: #dip into bag of all possible moves

            if (curr_dist_to_pred <= 3): #if predator too close and enough steps in the bag then run away from predator
                moves_sorted = sorted(all_possible_moves, key=lambda t: (-t[1], t[0]))
                move_choice=[]
                for move in moves_sorted: #to check if there are multiple /best/ neighbours which have the same distance to prey and predator
                    if (moves_sorted[0][0] == move[0] and moves_sorted[0][1] == move[1]):
                        move_choice.append(move)
                neighbour_choice = random.choice(move_choice) #selecting from these multiple neighbours at random
                agent_pos = neighbour_choice[2]

            else: #if step count exceeds this threshold, irrespective of predator distance make a dash to the prey
                moves_sorted = sorted(all_possible_moves, key=lambda t: (t[0], -t[1]))
                move_choice=[]
                for move in moves_sorted: #to check if there are multiple /best/ neighbours which have the same distance to prey and predator
                    if (moves_sorted[0][0] == move[0] and moves_sorted[0][1] == move[1]):
                        move_choice.append(move)
                neighbour_choice = random.choice(move_choice) #selecting from these multiple neighbours at random
                agent_pos = neighbour_choice[2]
        
        path.append(agent_pos)

        steps += 1

        if (agent_pos == prey_4.pos): #if agent has already moved to location of prey
            break
        else: #if prey not caught, treat as survey and perform another belief update
            prey_not = 1-belief_vector[agent_pos-1] #incase we dont find, make probability at that position 0 and divide entire array by probability of prey not being in that position
            belief_vector[agent_pos-1] = 0
            for i in range(len(belief_vector)):
                belief_vector[i] = belief_vector[i]/prey_not
            
        #print(str(sum(belief_vector)) + " Update 2 Timestep: " + str(steps))
        #print(str(belief_vector) + ' first update during timestep')
        #print(belief_vector)
        
        #if (sum(belief_vector) > 1.01 or sum(belief_vector) < 0.99):
        #    fail_list.append((2,steps))        
        #print(sum(belief_vector))
        """quasi_3 = sum(belief_vector)
        #print(quasi)
        for i in range(len(belief_vector)):
            belief_vector[i] /= quasi_3 #to ensure that belief vector sums up to 1, leaving no margin for error"""
            

        #move predator and prey
        prey_4.prey_move(g)

        if (prey_4.pos == agent_pos): #if prey luckily moves to location of agent
            break

        predator_4.pred_move(g, agent_pos)

        if(predator_4.pos == agent_pos): #if predator catches agent
            fail = 1
            break
        
        #belief update for next timestep
        temp_belief_vector = belief_vector.copy() #making a temporary belief vector for correct calculations
        for i in range(len(belief_vector)):
            mag_sum = temp_belief_vector[i]/(len(g.graph[i+1])+1) #probability of prey staying in same position
            i_neighbours = g.graph[i+1] #getting neighbours of current position
            #print(g.graph)
            #print(i+1)
            #print(i_neighbours)
            for bloke in i_neighbours:
                mag_sum = mag_sum + (temp_belief_vector[bloke-1]/(len(g.graph[bloke])+1)) #probability of prey moving to node i from node bloke
            belief_vector[i] = mag_sum #assigning probability
        #print(str(belief_vector) + ' last update before next timestep')
        #print(str(sum(belief_vector)) + " Update 3 Timestep: " + str(steps))
        #print(belief_vector)
        #if (sum(belief_vector) > 1.01 or sum(belief_vector) < 0.99):
        #    fail_list.append((3,steps))        
        """quasi_2 = sum(belief_vector)
        print(quasi_2)
        for i in range(len(belief_vector)):
            belief_vector[i] /= quasi_2"""
        #print(belief_vector)

    #if (len(fail_list) > 0):
    #    print("Fail List: " + str(fail_list))

    if(steps > 1000):
            fail = 2     
    return fail, path

In [10]:
wins = 0
losses = 0
timeout = 0
times = 0
win_dist = []
loss_dist = []

while times < 3000:
    g = Graph()
    graph = g.addEdge()
    while graph:
        graph = g.addEdge()
    res, path = Agent4(g)
    
    if(res == 0):
        wins += 1
        win_dist.append(len(path))
    elif(res == 1):
        losses += 1
        loss_dist.append(len(path))
    else:
        timeout += 1
    times += 1
    
print('Number of wins: ' + str(wins))
print("Max Steps Taken During a Win: " + str(max(win_dist)-1))
print("Average Steps Taken During a Win: " + str((sum(win_dist)/wins)-1))
print('Number of losses: ' + str(losses))
print("Max Steps Taken During a Loss: " + str(max(loss_dist)-1))
print("Average Steps Taken During a Loss: " + str((sum(loss_dist)/losses)-1))
print('Number of timeouts: ' + str(timeout))
print("Success Rate: " + str((wins/times)))

Number of wins: 2972
Max Steps Taken During a Win: 170
Average Steps Taken During a Win: 25.21298788694482
Number of losses: 28
Max Steps Taken During a Loss: 106
Average Steps Taken During a Loss: 27.571428571428573
Number of timeouts: 0
Success Rate: 0.9906666666666667
